In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [2]:
!python -V

Python 3.10.14


In [3]:
import os
import pickle
import pandas as pd

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
year = 2023
month = 3
df = read_data(f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet")

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [8]:
print(f"The standard deviation of the predicted duration is {y_pred.std()}")

The standard deviation of the predicted duration is 6.247488852238703


In [9]:
df_result = pd.DataFrame()
ride_id = f"{year:04d}/{month:02d}_" + df.index.astype("str")

df_result["ride_id"] = ride_id
df_result["duration"] = y_pred

df_result.head()

,ride_id,duration
0,2023/03_0,16.245906
1,2023/03_1,26.134796
2,2023/03_2,11.884264
3,2023/03_3,11.997720
4,2023/03_4,10.234486


In [10]:
output_file_name = f"prediction_yellow_tripdata_{year:04d}_{month:02d}.parquet"

df_result.to_parquet(
    output_file_name,
    engine="pyarrow",
    compression=None,
    index=False,
)

In [11]:
file_size = os.path.getsize(output_file_name) / (1024 ** 2)

print(f"The size of the output file is {file_size} MB")

The size of the output file is 65.46185111999512 MB
